In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

##Ошибка в методе   
    def forward(self, x, mask):
        ff_output = self.feed_forward(x)
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

## Ошибка в методе
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        nopeak_mask = nopeak_mask.to(tgt_mask.device)  # Move nopeak_mask to the same device as tgt_mask
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask


    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
src_vocab_size = 1000
tgt_vocab_size = 1000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 50
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [ ]:
from random import randint

In [ ]:
def generate_sample():
    enc_src_start = randint(1, 944)
    dec_src_start = randint(1, 944)

    enc_src = torch.arange(enc_src_start, enc_src_start + 50, dtype=torch.int64)
    dec_src = torch.arange(dec_src_start, dec_src_start + 50, dtype=torch.int64)

    trg = torch.arange(dec_src_start + 1, dec_src_start + 51, dtype=torch.int64)  # Пример для целевой последовательности

    return enc_src, dec_src, trg

In [ ]:
def generate_batch(batch_size: int = 128):
    src_batch = []
    dec_batch = []
    trg_batch = []

    while len(src_batch) < batch_size:
        src_sample, dec_sample, trg_sample = generate_sample()
        src_batch.append(src_sample.unsqueeze(0))
        dec_batch.append(dec_sample.unsqueeze(0))
        trg_batch.append(trg_sample.unsqueeze(0))

    src_batch = torch.cat(src_batch, dim=0)
    dec_batch = torch.cat(dec_batch, dim=0)
    trg_batch = torch.cat(trg_batch, dim=0)

    return src_batch, dec_batch, trg_batch

In [ ]:
transformer = transformer.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for step in tqdm(range(1000)):
    src_batch, dec_batch, trg_batch = generate_batch(64)
    src_batch, dec_batch, trg_batch = src_batch.to(device), dec_batch.to(device),trg_batch.to(device)
    optimizer.zero_grad()
    output = transformer(src_batch, dec_batch)
    loss = criterion(output.view(-1, output.size(-1)), trg_batch.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Step: {step+1}, Loss: {loss.item()}")

  0%|          | 1/1000 [00:01<28:07,  1.69s/it]

Step: 1, Loss: 7.090574264526367


  0%|          | 2/1000 [00:01<14:08,  1.18it/s]

Step: 2, Loss: 7.012030124664307


  0%|          | 3/1000 [00:02<09:41,  1.71it/s]

...

Step: 998, Loss: 1.1642241588560864e-05


100%|█████████▉| 999/1000 [05:01<00:00,  3.30it/s]

Step: 999, Loss: 1.2179474651929922e-05


100%|██████████| 1000/1000 [05:02<00:00,  3.31it/s]

Step: 1000, Loss: 1.1905620340257883e-05


In [ ]:
transformer.eval()
src_sample = torch.zeros(50, dtype=torch.int64)
src_sample[:10] = torch.arange(833, 843, dtype=torch.int64)

In [ ]:
src_sample.unsqueeze(0)

tensor([[833, 834, 835, 836, 837, 838, 839, 840, 841, 842,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0]])

In [ ]:
res = transformer(src_sample.unsqueeze(0).to(device), src_sample.unsqueeze(0).to(device))

In [ ]:
res.squeeze().argmax(dim=1)

tensor([834, 835, 836, 837, 838, 839, 840, 841, 842, 843,  47,  47,  47,  47,
         47,  47,  47,  47,  47,  47,  47,  47,  47,  47,  47,  47,  47,  47,
         47,  47,  47,  47,  47,  47, 250, 250, 250, 250, 250, 250, 250, 250,
        250, 250,  47,  47,  47, 250, 250, 250], device='cuda:0')

In [ ]:
# Вспомогательная функция для генерации последовательности по заданным параметрам
def generate_sequence(start, length):
    return torch.arange(start, start + length, dtype=torch.int64)

def evaluate_single_sequence(transformer):
    enc_src = generate_sequence(1, 50)
    dec_src = generate_sequence(50, 50)
    enc_src_tensor = enc_src.unsqueeze(0)  # Добавляем размерность пакета
    dec_src_tensor = dec_src.unsqueeze(0)  # Добавляем размерность пакета

    enc_src_tensor, dec_src_tensor = enc_src_tensor.to(device), dec_src_tensor.to(device)

    # Пропускаем входные последовательности через модель
    output = transformer(enc_src_tensor, dec_src_tensor)

    # Генерируем таргет
    trg = generate_sequence(51, 50)

    # Конвертируем тензор вывода в последовательность
    predicted_sequence = torch.argmax(output, dim=-1).squeeze().tolist()

    # Выводим входную, предсказанную и целевую последовательности
    print("Входная последовательность (enc_src):", enc_src)
    print("Входная последовательность (dec_src):", dec_src)
    print("Предсказанная последовательность:", predicted_sequence)
    print("Целевая последовательность:", trg)
    print("=" * 50)

# Вызов функции оценки для отдельной последовательности
evaluate_single_sequence(transformer)


Входная последовательность (enc_src): tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])
Входная последовательность (dec_src): tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
        86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])
Предсказанная последовательность: [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Целевая последовательность: tensor([ 51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  